In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

In [0]:
df_anz = spark.table("budget_cata.silver.anz")
df_manual = spark.table("budget_cata.silver.manual_transactions")

In [0]:
# Create master table by unioning all the silver bank tables, incl. manual_transactions
df_master = df_anz.unionByName(df_manual, allowMissingColumns=True)
df_master = df_master.unionByName(spark.table("budget_cata.silver.commbank"), True)
df_master = df_master.unionByName(spark.table("budget_cata.silver.westpac"), True)

In [0]:
df_master = (
    df_master
    .select(
        col("Date").alias("transaction_date"),
        col("Type").alias("transaction_type"),
        col("category"),
        col("note"),
        col("Amount").alias("transaction_amount"),
        col("cleaned_desc").alias("transaction_details"),
        col("Description").alias("transaction_details_raw"),
        col("location"),
        col("bank"),
        col("payment"),
        col("transaction_id"),
        col("updated_at"),
    )
)


In [0]:
(df_master.write
 .format("delta")
 .mode("append")
 .saveAsTable("budget_cata.gold.master_stg"))